In [1]:
import numpy as np
import matplotlib.pyplot as plt
import ctypes
from scipy.stats import chi2
from statsmodels.sandbox.stats.multicomp import fdrcorrection_twostage as fdr
import os
import argparse
import time
from iotools.dosageparser import DosageParser
from iotools.readVCF import ReadVCF 
from scipy.optimize import minimize
from scipy.special import erfinv
#from inference.regulizer_optimizer import OptimizeRegularizer
from inference.per_snp_optimiser import OptimizeRegularizer
#from inference.perm_regulizer_optimizer import OptimizeRegularizer
#from select_genes_per_snp import linear_reg
import regstat
import regnull
import hist_qq
#from qqplot import plot_hist_qq

In [2]:
def logml(X, Y, sigmabeta, sigmax):
    nsnps = X.shape[0]
    nsamples = X.shape[1]
    sigmabeta2 = sigmabeta * sigmabeta
    sigmax2    = np.square(sigmax)
    
    Yt = Y.T # shape N x G
    Abase = sigmabeta2 * np.dot(Yt, Yt.T)
    
    totres = 0
    for i in range(nsnps):
        A = Abase.copy()
        A[np.diag_indices(nsamples)] += sigmax2[i]
        logdetA = np.linalg.slogdet(A)
        invA = np.linalg.inv(A)
        res = - 0.5 * ( nsamples * np.log(2 * np.pi) + logdetA[1] + np.dot(X[i, :].T, np.dot(invA, X[i, :])) )
        totres += res
    return totres

In [3]:
def permuted_dosage(expr, nsnp = 5000, fmin = 0.1, fmax = 0.9, maketest = False):

    f = np.random.uniform(fmin, fmax, nsnp)
    if maketest:
        f = np.repeat(0.1, nsnp)
    nsample = expr.shape[1]

    dosage = np.zeros((nsnp, nsample))
    for i in range(nsnp):
        if maketest:
            nfreq = np.array([[279,  54,   5]])[0]
        else:
            mafratios = np.array([(1 - f[i])**2, 2 * f[i] * (1 - f[i]), f[i]**2])
            nfreq  = np.random.multinomial(nsample, mafratios, size=1)[0]
        f1 = np.repeat(0, nfreq[0])
        f2 = np.repeat(1, nfreq[1])
        f3 = np.repeat(2, nfreq[2])
        x  = np.concatenate((f1,f2,f3))
        dosage[i, :] = np.random.permutation(x)

    maf2d = f.reshape(-1, 1)
    gtnorm = (dosage - (2 * maf2d)) / np.sqrt(2 * maf2d * (1 - maf2d))
    gtcent = dosage - np.mean(dosage, axis = 1).reshape(-1, 1)

    return gtnorm, gtcent


def Expression(gt, gx, beta, cfrac = 0.001):

    ntrans  = gt.shape[0]
    ngene   = gx.shape[0]
    nsample = gx.shape[1]

    liabilities = np.zeros((ngene, nsample))
    cindex = np.zeros((ntrans, ngene))                                           # Index matrix of gene / trans-eQTLs pairs
    nc  = np.random.gamma(ngene * cfrac, scale = 1.0, size = ntrans).astype(int) # number of target genes for each trans-eQTL
    for i in range(ntrans):
        ncausal = min(ngene, nc[i])                                              # do something better, trans-eQTLs cannot target all genes
        choose = np.random.choice(ngene, ncausal, replace = False)
        cindex[i, choose] = 1                                                    # mark these genes as causal

    gtarget = list()

    for i in range(ngene):
        csnps = np.where(cindex[:, i] == 1)[0]
        if csnps.shape[0] > 0: # then we got a trans-eQTL
            betas = np.random.normal(0, beta, size = csnps.shape[0])
            #betas *= np.sqrt( H2[i] / np.sum(np.square(beta)) )
            liabilities[i, :] = np.dot(gt[csnps, :].T, betas)
            gtarget.append(i)

    newGX = gx + liabilities
    newGX = (newGX - np.mean(newGX, axis = 1).reshape(-1, 1)) / np.std(newGX, axis = 1).reshape(-1, 1)
    return newGX, nc
### Simulation of data

def simulate(sb, sx):
    ngene = 1000
    nsample = 338
    nsnp = sx.shape[0]
    
    betas = np.random.normal(0, sb, ngene)
    Y = np.zeros((ngene, nsample))
    for i in range(ngene):
        Y[i, :] = np.random.normal(0, 1, nsample)

    Y = (Y - np.mean(Y, axis = 1).reshape(-1, 1)) / np.std(Y, axis = 1).reshape(-1, 1)
    yTbeta = np.dot(Y.T, betas)
    X = np.zeros((nsnp, nsample))
    for i in range(nsnp):
        error = np.random.normal(0, sx[i], nsample)
        X[i, :] = yTbeta + error
    return X, Y
def read_expression(filename):
    gene_names = list()
    expression = list()
    with open(filename, 'r') as genefile:
        header = genefile.readline()
        donorids = header.strip().split("\t")[1:]
        for line in genefile:
            linesplit = line.strip().split("\t")
            expression.append(np.array(linesplit[1:], dtype=float))
            gene_names.append(linesplit[0])
    expression = np.array(expression)
    return donorids, expression, gene_names


'''def norm_binom(genotype, f):
    genotype = (genotype - (2 * f)) / np.sqrt(2 * f * (1 - f))
    return genotype
'''

def parse_geno (genotype_filename, sample_filename, startsnp, endsnp):                              #Read genotype here
    ds        = DosageParser(genotype_filename, sample_filename, startsnp, endsnp)
    dosage    = ds.dosage
    snpinfo   = ds.snpinfo
    donorids  = ds.sample_id
    nsnps     = ds.nsnps
    nsample   = ds.nsample
    return dosage, snpinfo, donorids, nsnps, nsample

                                                                                                    #quality check, matching
def quality_check ( sampleids, donorids):
    choose_ids     = [x for x in donorids if x in sampleids]
    #dosage_indices = [i for i, x in enumerate(donorids)  if x in choose_ids]
    #exprsn_indices = [i for i, x in enumerate(sampleids) if x in choose_ids]
    dosage_indices = [donorids.index(i) for i in choose_ids]
    exprsn_indices = [sampleids.index(i) for i in choose_ids]
    return dosage_indices, exprsn_indices

def read_distance(filename):
    samplelist = list()
    distances = list()
    with open(filename, 'r') as mfile:
        mfile.readline()
        for line in mfile:
            linestrip = line.strip().split("\t")
            samplelist.append(linestrip[0])
            distances.append(np.array(linestrip[1:],dtype=float))
    return np.array(distances), samplelist

In [8]:
# ### Simulation of data

# ngenes = 1000
# nsamples = 200
# nsnps = 10
# true_sigmabeta = 0.02#0.0006
# nmin = 1
# cfrac = 1
# nmax = int(ngenes)
# ncausal = np.random.randint(nmin, nmax, nsnps)
# fmin = 0.1
# fmax = 0.5
# ntrans = 5

# Y = np.random.randn(ngenes * nsamples).reshape(ngenes, nsamples)

# gtnorm, gtcent = permuted_dosage(Y, nsnp = nsnps, fmin = fmin, fmax = fmax)
       
# # Trans-eQTL
# if ntrans > 0:
#     newgx,  nc = Expression(gtnorm[-ntrans:, :], Y, beta=true_sigmabeta, cfrac = cfrac)
# Y = newgx

# # X = np.zeros((nsnps, nsamples))
# # for i in range(nsnps):
# #     choose = np.random.choice(ngenes, ncausal[i], replace=False)
# #     betas = np.random.normal(0, true_sigmabeta, ncausal[i])
# #     #betas = np.random.normal(0, true_sigmabeta, ngenes)
# #     #choose = np.arange(ngenes)
# #     X[i, :] = np.dot(Y[choose, :].T, betas) + np.random.normal(0, 1, nsamples)

# # sigmax = np.linspace(0, 0.4, nsnps)
# # X, Y = simulate(true_sigmabeta, sigmax)
# # gtcent = X - np.mean(X, axis = 1).reshape(-1, 1)

In [11]:
#out_fileprefix      = 
genotype_filename   = "data/geno/GTEx_450Indiv_genot_imput_info04_maf01_HWEp1E6_dbSNP135IDs_donorIDs_dosage_chr2.gz"
sample_filename     = "data/geno/donor_ids.fam"
#expression_filename = "/home/raktim/Documents/tejaas/final/PCA_KNN_Correction/main/expr2.txt"
expression_filename = "Whole_Blood_Analysis.v6p.normalized.expression.txt"

startsnp            = 1
endsnp              = 1000
optim               = 0
#user_sigbeta        = 0.001636581825407
#transgeno           = "data/GTEx_450Indiv_genot_imput_info04_maf01_HWEp1E6_dbSNP135IDs_donorIDs_dosage_filteredTEJAAS_transeqtls.gz"#opts.trans_genofile
sampleids, expression, gene_names = read_expression(expression_filename)
dosage, snpinfo, donorids, nsnps, nsample = parse_geno ( genotype_filename, sample_filename, startsnp, endsnp)
dosage_indices, exprsn_indices = quality_check (sampleids , donorids)
conv_dosage = np.round(dosage)
expr = expression[:, exprsn_indices]
#expr = np.random.normal(0, 1, 23973 * 338).reshape(23973, 338)
geno = conv_dosage[:, dosage_indices]
Y = (expr - np.mean(expr, axis = 1).reshape(-1, 1)) / np.std(expr, axis = 1).reshape(-1, 1)
Y = Y[:500]
gtcent = geno - np.mean(geno, axis = 1).reshape(-1, 1)
print ("Completed data loading and processing\n")


low maf  poly alleles  complement alleles 

402   64   187 

Total SNPs read in :  1000
Total SNPs remained :  347

Completed data loading and processing



In [12]:
# import numpy as np
# from scipy.optimize import minimize, fmin_l_bfgs_b
# import regstat
# from tqdm import tqdm
# import qstats.crrstat as crrstat



# class OptimizeRegularizer:

#     def __init__(self, genotype, gene_expr, sigmax = 1, tol = 0.01, sigmabeta = 2):
#         self._genotype = genotype
#         self._expr = gene_expr
#         self._sigmax = sigmax
#         self._sigmareg = sigmabeta
#         self._tolerance = tol
#         self._niter = 0
#         self._U, self._S, self._vt = self._svd(self._expr)

#         self._A =  np.dot(self._expr,self._expr.T)
#         self._detA = np.linalg.det(self._A)
#         self._trA  = np.trace(self._A)
#         self._trA_2 = np.trace(self._A)**2
#         self._tr_A2 = np.trace(np.dot(self._A,self._A))

#     def _svd(self, expr):
#         U, S, Vt = np.linalg.svd(np.transpose(expr),full_matrices=False)
#         return (U, S, Vt)

#     @property
#     def sigmareg(self):
#         return self._sigmareg


#     @property
#     def niter(self):
#         return self._niter

#     def _logml(self, logsigmab, *args):
#         sigmabeta = np.e ** logsigmab
#         X, Y, S, U, sigmax = args
#         nsnps      = X.shape[0]
#         nsamples   = X.shape[1]
#         ngenes     = Y.shape[0]
#         sigmabeta2 = sigmabeta * sigmabeta
#         sigmax2    = sigmax**2
#         yty = np.dot(Y, Y.T)
#         lml = 0
#         for snp in range(nsnps):
#             #A = yty.copy()
#             #print(np.linalg.slogdet(A)[1])
#             #A[np.diag_indices(ngenes)] += sigmax2[snp] / sigmabeta2
#             #logdetA = np.linalg.slogdet(A)[1]
#             #eps = sigmabeta2/sigmax2[snp]
#             #log_identity_term = - np.log(eps) * ngenes
#             #logdetA = log_identity_term + np.log(1 + self._detA + eps * self._trA + 0.5 * eps * eps * self._trA_2 - 0.5 * eps * eps *self._tr_A2)

#             Smod = np.diag(np.square(S) / (np.square(S) + sigmax2[snp] / sigmabeta2))
#             logdetA = np.sum(np.log(np.square(S) + sigmax2[snp] / sigmabeta2) ) + (ngenes - nsamples)*np.log(sigmax2[snp] / sigmabeta2)
#             #print(logdetA)
#             W = np.dot(U, np.dot(Smod, U.T))
#             const_term = - 0.5 * ngenes * np.log(2 * np.pi * sigmabeta2) - 0.5 * logdetA
#             snp_term = 0.5 * np.dot(X[snp, :].T, np.dot(W, X[snp,:])) / sigmax2[snp]
#             lml = lml + (const_term) + snp_term
#         #print(-lml)
#         return -lml

#     def _grad_logml(self, logsigmab, *args):
#         sigmabeta = np.e ** logsigmab          #x is logsigmabeta
#         X, Y, S, U, sigmax = args
#         nsnps = X.shape[0]
#         nsamples = X.shape[1]
#         ngenes = Y.shape[0]
#         sigmabeta2 = sigmabeta * sigmabeta
#         sigmax2    = sigmax**2

#         term1 = -ngenes



#         der = 0
#         for i in range(nsnps):
#             Smod = np.square(S) * sigmabeta2 / sigmax2[i]
#             term2 = (ngenes - nsamples) + np.sum(1 / (Smod + 1))
#             term3 = 0
#             for k in range(nsamples):
#                 uk = U[:, k]
#                 sk = S[k]
#                 smod = sk * sk * sigmabeta2 / sigmax2[i]
#                 term3 += smod * np.square(np.dot(uk, X[i,:])) / sigmax2[i] / np.square(smod + 1)
#             der += term1 + term2 + term3
#         return der

#     def update (self):
#         sigmareg_old = np.e ** self._sigmareg
#         N = 10                                                                                         # expected number of true trans-eQTLs. Hard-coded
#         iterate = True
#         sigmax2 = self._sigmax ** 2
#         logsb = 2
#         arguments = ()
#         def call(x):
#             print(x)
#         while iterate:
#             Qsvd = np.array(range(self._genotype.shape[0]))
#             #for snp in tqdm(range(self._genotype.shape[0])):
#             #Q, _, _  = regstat.rscore(self._genotype[snp:snp+1,:], self._expr, sigmareg_old**2, np.array([sigmax2[snp]])) 
#             _, Q, _, _ = crrstat.perm_null(self._genotype, self._expr, np.array([sigmareg_old**2]*len(Qsvd)), sigmax2)
#             Qsvd = Q
#             #print(np.argsort(-Qsvd)[0:N])
#             #print(Qsvd[0:N])
#             top_Q_indx = np.argsort(-Qsvd)[0:N]

#             top_geno = self._genotype[top_Q_indx,:]
#             arguments = (top_geno, self._expr, self._S, self._U, self._sigmax[top_Q_indx])

#             print("optimizing...")
#             #print(self._logml(self._sigmareg, *arguments))
#             #res = fmin_l_bfgs_b(self._logml, [2], fprime = self._grad_logml, args = arguments)# jac=self._grad_logml)
#             res = minimize(self._logml, [logsb], method="BFGS", args = arguments, options={'disp': True}, callback=call)
#             #res = minimize(self._logml, [20], method="nelder-mead", args = arguments) 
#             #print(res)
#             logsb = res.x[0]
#             sigbeta_new = np.e**res.x[0]
#             print("current sigmabeta", sigbeta_new)

#             checksigma = self.check_convergence(sigbeta_new, sigmareg_old)

#             if checksigma:
#                 iterate = False
#                 sigma_optimized = sigbeta_new

#             self._niter += 1
#             sigmareg_old = sigbeta_new

#         self._sigmareg = sigma_optimized
#     def check_convergence(self, x, xold):
#         check = False
#         tol = self._tolerance
#         diff = x - xold
#         if diff == 0:
#             check = True
#         if not check and xold != 0.0:
#             diff_percent = abs(diff) / xold
#             if diff_percent < tol:
#                 check = True

#         return check

In [13]:
print(Y.shape)

(500, 338)


In [15]:
optim = True
sigbeta = 0.6
if optim:
    print ("\nSigma beta optimization started. Reading from the provided trans-genotype file.")
    try:
        sigmax = np.std(gtcent - np.mean(gtcent, axis = 0),axis = 1)
        #optimize_sigbeta   = OptimizeRegularizer(geno[:10], expr, sigmax = np.std(geno[:10].T,axis = 0))
        optimize_sigbeta   = OptimizeRegularizer(gtcent, Y, sigmax = sigmax)
        l = optimize_sigbeta.update()
        optimize_sigbeta.niter
        sigbeta            = optimize_sigbeta.sigmareg
        #print(sigbetas)
        
        print ("Optimized sigma beta value is:" , np.array(sigbeta),"\n")
        #del geno, conv_dosage, trans_dosage
    except OSError as e:
        print("\n",e, ". Trans-eQTLs file not provided for Optimization\n")
        raise SystemExit


else:
    print("\n=============================================================")
    print("\nsigma beta optimization not requested")
    if user_sigbeta == 0.006:
        print ("\nsigma beta not provided. Using default value of 0.006")
    else:
        print("\nsigma beta value provided as : " , user_sigbeta)
    print("=============================================================")


  0%|          | 0/347 [00:00<?, ?it/s]


Sigma beta optimization started. Reading from the provided trans-genotype file.


100%|██████████| 347/347 [00:16<00:00, 20.62it/s]

Optimized sigma beta value is: [2.09044492e-05 1.46892451e-03 2.75052179e-03 8.77675856e-03
 5.72156549e-03 9.12740702e-03 3.17945328e-07 8.20740225e-03
 5.60340018e-03 9.24990981e-03 8.39056209e-03 8.17524700e-03
 1.63881913e-05 8.39056209e-03 9.71750347e-04 9.41995396e-03
 6.90568012e-03 2.94999633e-05 8.98434730e-03 4.97918069e-03
 8.01672505e-03 8.03212302e-03 1.04068145e-02 1.04068145e-02
 2.54751691e-03 3.22537431e-05 1.04068145e-02 1.04382682e-02
 3.30621070e-05 1.82796328e-05 1.63025474e-05 6.36065578e-03
 1.89091663e-03 1.79420302e-03 8.31279516e-04 5.26333282e-04
 6.50290053e-03 2.17519644e-05 2.31305779e-03 2.56359183e-05
 5.12886885e-03 2.56359183e-05 2.71599480e-05 2.56359183e-05
 5.87837500e-03 2.71599480e-05 5.78579350e-03 5.12886885e-03
 1.45709966e-06 7.30387979e-03 6.10462830e-03 3.10961214e-05
 2.28383078e-05 5.44623218e-03 6.10462830e-03 1.75642483e-05
 6.10462830e-03 6.10462830e-03 4.27370978e-03 6.10462830e-03
 5.51762093e-03 6.10462830e-03 2.51290741e-05 6.104628

In [ ]:
### Marginal log likelihood (different ways of calculation)
sigmax = np.std(gtcent,axis = 1)
optimize_sigbeta   = OptimizeRegularizer(gtcent, Y, sigmax)

sigmabeta = np.logspace(-10, 4, 50, base = np.e)

U, S, Vt = np.linalg.svd(Y.T)

A =  np.dot(Y, Y.T)
detA = np.linalg.det(A)
trA  = np.trace(A)
trA_2 = np.trace(A)**2
tr_A2 = np.trace(np.dot(A,A))
args = (gtcent, Y, S, U, sigmax)
ysimp    = [optimize_sigbeta._logml(np.log(sbeta), *args) for sbeta in sigmabeta]
ycomp    = [logml(gtcent, Y, sbeta, sigmax) for sbeta in sigmabeta]
#ycompsvd = [logml_appr_old(gtcent, Y, sbeta, sigmax, detA, trA, trA_2, tr_A2) for sbeta in sigmabeta]

In [ ]:
sigmabeta = np.logspace(-10, 4, 50, base = np.e)
true_alpha = np.log(true_sigmabeta)
#print(true_sigmabeta)

fig = plt.figure(figsize = (6, 6))
ax1 = fig.add_subplot(111)

ax1.scatter(np.log(sigmabeta), -np.array(ysimp),    color = 'blue', label = 'Optim_lml')
ax1.plot(np.log(sigmabeta),    ycomp,    color = 'green', label = 'Complex')
#ax1.scatter(np.log(sigmabeta), ycompsvd, color = 'salmon', label = 'Complex SVD')#'salmon'
print(true_alpha)
#ax1.axvline(x = true_alpha, color = 'red', label = 'True')


ax1.axvline(x = np.log(sigbeta), color = 'blue', label = 'Optim_result')

plt.legend(loc = 'upper left')
plt.show()

In [ ]:
U, S, Vt = np.linalg.svd(np.transpose(Y),full_matrices=False)
print(Y.shape)
print(S.shape)

In [ ]:
YtY = np.dot(Y, Y.T)
print(YtY.shape)
YtY[np.diag_indices(Y.shape[0])] += 5 #sigmax2[snp] / sigmabeta2

print(np.linalg.slogdet(YtY))
#print(np.product(S**2))
print(np.sum(np.log(S**2 + 5)) + (Y.shape[0] - Y.shape[1])*np.log(5) )